In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My Drive/google_colab_gpu/GSOC 2020/CERN-HSF

/content/drive/My Drive/google_colab_gpu/GSOC 2020/CERN-HSF


In [3]:
ls

cifar10vgg.h5          model3.hdf5
ground_truth.csv       model3.json
Keras_CERN.ipynb       modelVGG.hdf5
Keras_inference.ipynb  modelVGG.json
Keras_model2.ipynb     output.csv
model1.h5              pytorch1.pt
model1.hdf5            Pytorch_CERN.ipynb
model1.json            SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
model2.h5              SingleElectronPt50_IMGCROPS_n249k_RHv1_inference.hdf5
model2.hdf5            SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5
model2.json            SinglePhotonPt50_IMGCROPS_n249k_RHv1_inference.hdf5


In [0]:
#import cv2
import numpy as np
import pandas as pd
#from google.colab.patches import cv2_imshow
import h5py

In [0]:
#import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#import cv2
#from keras.datasets import mnist

#from keras.utils import np_utils
#from keras.models import Sequential,load_model
#from sklearn.model_selection import train_test_split
import torch.nn
#import torchvision.datasets as dsets
#import torchvision.transforms as transforms
#from torch.autograd import Variable

In [0]:
filename='SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
data1 = h5py.File(filename, 'r')
Y1=data1['y']
X1=data1['X']

In [0]:
filename='SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
data0 = h5py.File(filename, 'r')
Y0=data0['y']
X0=data0['X']

In [0]:
X_final=np.concatenate((X0[:],X1[:]),axis=0)
Y_final=np.concatenate((Y0[:],Y1[:]),axis=0)

In [0]:
X_train,X_valid, Y_train, Y_valid = train_test_split(X_final,Y_final,test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)

(398400, 32, 32, 2) (398400,)
(99600, 32, 32, 2) (99600,)


In [0]:
X_train0=(X_train[:,:,:,0].reshape((X_train.shape[0],1,X_train.shape[1],X_train.shape[2])))
X_valid0=(X_valid[:,:,:,0].reshape((X_valid.shape[0],1,X_valid.shape[1],X_valid.shape[2])))
X_train1=(X_train[:,:,:,1].reshape((X_train.shape[0],1,X_train.shape[1],X_train.shape[2])))
X_valid1=(X_valid[:,:,:,1].reshape((X_valid.shape[0],1,X_valid.shape[1],X_valid.shape[2])))
X_train.shape,X_valid.shape,X_train0.shape,X_valid0.shape,X_train1.shape,X_valid1.shape

((398400, 32, 32, 2),
 (99600, 32, 32, 2),
 (398400, 1, 32, 32),
 (99600, 1, 32, 32),
 (398400, 1, 32, 32),
 (99600, 1, 32, 32))

In [0]:
X_train0, Y_train, X_valid0, Y_valid, X_train1, X_valid1 = map(torch.tensor, (X_train0, Y_train_tp, X_valid0, Y_valid_tp, X_train1, X_valid1))

In [0]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, inputs, labels):
        'Initialization'
        self.labels = labels
        self.inputs = inputs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.inputs)

  def __getitem__(self, index):
        'Generates one sample of data'

        # Get data and get label
        X = self.inputs[index]
        y = self.labels[index]

        return X, y

In [0]:
train_loader = torch.utils.data.DataLoader(dataset={'input':X_train0,'output':Y_train},
                                          batch_size=1024,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset={'input':X_valid0},
                                          batch_size=1024,
                                          shuffle=False)

In [0]:
window_height=32
window_width=32

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        #self.relu1 = nn.Relu()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        #self.relu2 = nn.Relu()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*8*8, 128)
        #self.relu3 = nn.Relu()
        self.fc2 = nn.Linear(128, 64)
        #self.relu4 = nn.Relu()
        self.fc3 = nn.Linear(64, 1)
        #self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x


net = Net()
net.cuda()
summary(net, (1, 32, 32), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
         MaxPool2d-2           [-1, 16, 16, 16]               0
            Conv2d-3           [-1, 32, 16, 16]           4,640
         MaxPool2d-4             [-1, 32, 8, 8]               0
            Linear-5                  [-1, 128]         262,272
            Linear-6                   [-1, 64]           8,256
            Linear-7                    [-1, 1]              65
Total params: 275,393
Trainable params: 275,393
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.24
Params size (MB): 1.05
Estimated Total Size (MB): 1.29
----------------------------------------------------------------


In [0]:
use_cuda = True

In [0]:
if use_cuda and torch.cuda.is_available():
    net.cuda()

In [0]:
batch_size=1024
import torch
from torch.utils import data
#import cudnn
#from my_classes import Dataset


# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True

# Parameters
params = {'batch_size': batch_size,
          'shuffle': True,
          }
max_epochs = 100

# Generators
training_set = Dataset(X_train0, Y_train)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(X_valid0, Y_valid)
validation_generator = data.DataLoader(validation_set, **params)

In [0]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [0]:
num_epochs=10
correct=0
total=0
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(training_generator):   # Load a batch of images with its (index, data, class)
        #images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        #labels = Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(inputs)
                                      # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        outputs[outputs>0.5]=1
        outputs[outputs<0.5]=0
        total += labels.shape[0]                    # Increment the total count
        correct += (outputs == labels).sum()
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.4f'
                 %(epoch+1, num_epochs, i+1, len(X_train0)//batch_size, loss.item(), ((100 * correct.item() / total))))
    #outputs = net(X_valid0.cuda())                             # Forward pass: compute the output class given a image
    #loss = criterion(outputs, Y_valid.cuda())

Epoch [1/10], Step [100/389], Loss: 0.5880, Accuracy: 72.3496
Epoch [1/10], Step [200/389], Loss: 0.5885, Accuracy: 72.2695
Epoch [1/10], Step [300/389], Loss: 0.5496, Accuracy: 72.4202
Epoch [2/10], Step [100/389], Loss: 0.5557, Accuracy: 72.4920
Epoch [2/10], Step [200/389], Loss: 0.5232, Accuracy: 72.4929
Epoch [2/10], Step [300/389], Loss: 0.5393, Accuracy: 72.4763
Epoch [3/10], Step [100/389], Loss: 0.5464, Accuracy: 72.4199
Epoch [3/10], Step [200/389], Loss: 0.5709, Accuracy: 72.4400
Epoch [3/10], Step [300/389], Loss: 0.5871, Accuracy: 72.4235
Epoch [4/10], Step [100/389], Loss: 0.5619, Accuracy: 72.4482
Epoch [4/10], Step [200/389], Loss: 0.5391, Accuracy: 72.4549
Epoch [4/10], Step [300/389], Loss: 0.5510, Accuracy: 72.4521
Epoch [5/10], Step [100/389], Loss: 0.5422, Accuracy: 72.4495
Epoch [5/10], Step [200/389], Loss: 0.5580, Accuracy: 72.4501
Epoch [5/10], Step [300/389], Loss: 0.5097, Accuracy: 72.4626
Epoch [6/10], Step [100/389], Loss: 0.5450, Accuracy: 72.4890
Epoch [6

In [0]:
correct = 0
total = 0
for inputs, labels in validation_generator:
    #inputs = Variable(images.view(-1, 28*28))
    
    if use_cuda and torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
    
    
    outputs = net(inputs)
    outputs[outputs>0.5]=1
    outputs[outputs<0.5]=0
    total += labels.shape[0]                    # Increment the total count
    correct += (outputs == labels).sum()     # Increment the correct count
    
print('Validation Accuracy of the network on the ' + str(total)+' test images: %.4f' % ((100 * correct.item() / total))+' correct: '+str((correct).item()))

Accuracy of the network on the 99600 test images: 72.2791 correct: 71990


In [0]:
torch.save(net.state_dict(), 'pytorch1.pt')

In [0]:
net = Net()
net.load_state_dict(torch.load('pytorch1.pt'))
if use_cuda and torch.cuda.is_available():
    net.cuda()
net.eval()

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)